In [1]:
#pip install --pre mesa[viz]

In [2]:
# import libraries
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.animation import FuncAnimation
import mesa
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import numpy as np
import math
from IPython.display import HTML
from matplotlib import rc


This code runs with mesa version 3.00 or higher

In [3]:
# check mesa version
print("Mesa version: ", mesa.__version__)

Mesa version:  3.1.1


In [4]:
# Function to calculate Euclidean distance (distance between two points)
def euclidean_distance(pos1, pos2):
    # Use the formula to calculate distance between two coordinates
    return math.sqrt((pos1[0] - pos2[0]) ** 2 + (pos1[1] - pos2[1]) ** 2)

# Base Model

In [5]:
###################
### AGENT CLASS ###
###################
### I'm defining an agent class for the crew that are essentially walking signs
class CrewMember(Agent):
    def __init__(self, model, vision=5):
        super().__init__(model)
        self.previous_pos = None  # Keep track of previous position
        self.vision = vision # vision is range to detect a NavigationAgent
        self.type = 'Crew' ### add an atrivute to identify crew or passenger
    
    def move_randomly(self):
        #check if passenger (navigationAgent) is in vision range
        vision_area = self.model.grid.get_neighborhood(self.pos, moore=True, radius=self.vision, include_center=False)
        for cell in vision_area:
            for i in self.model.grid.get_cell_list_contents(cell): # i of list of agents in the given cell, that is in vision ares (defined in vision_area)
                if isinstance(i,NavigationAgent):
                    return  #stop moving if see passenger 
                    
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        # Filter steps to only those without obstacles and with fewer than 8 agents
        valid_steps = [step for step in possible_steps if step not in self.model.obstacles and len(self.model.grid.get_cell_list_contents(step)) < 8]
        if valid_steps:
            random_step = self.random.choice(valid_steps) # Randomly choose a valid position and move there
            self.model.grid.move_agent(self, random_step) # MESA `move_agent` function moves the agent to the chosen position

    def step(self):
        self.move_randomly()

# Define the NavigationAgent class
class NavigationAgent(Agent):
    def __init__(self, model, vision=5): # Default vision range of 5 cells
        super().__init__(model)  # MESA `Agent` class initialization, auto-assigns unique_id in Mesa 3.0
        # Attributes of each agent
        self.found_exit = False  # Track if agent has reached the exit
        self.previous_pos = None  # Previous position of the agent
        self.vision = vision  # Vision range of the agent
        self.type= 'Passenger'

    # Function to move the agent towards the exit
    def move_towards_exit(self):
        self.previous_pos = self.pos  # Store the current position before moving
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        
        min_distance = float('inf')  # Start with a very large distance
        best_step = None  # Initialize best step as None
        
        # Find the closest exit and the best step toward it
        for step in possible_steps:
            if step not in self.model.obstacles and len(self.model.grid.get_cell_list_contents(step)) < 8:
                for exit_location in self.model.exit_locations:
                    dist = euclidean_distance(step, exit_location)  # Distance to the exit
                    if dist < min_distance:
                        min_distance = dist
                        best_step = step  # Update best step to be closer to the exit
        
        if best_step:
            self.model.grid.move_agent(self, best_step)  # Move the agent to the best step


    # Function to move the agent randomly if the exit is not in sight
    def move_randomly(self):
        self.previous_pos = self.pos
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        
        # Filter steps to only those without obstacles and with fewer than 8 agents
        valid_steps = [step for step in possible_steps if step not in self.model.obstacles and len(self.model.grid.get_cell_list_contents(step)) < 8]
        
        if valid_steps:
            # Randomly choose a valid position and move there
            random_step = self.random.choice(valid_steps)
            # MESA `move_agent` function moves the agent to the chosen position
            self.model.grid.move_agent(self, random_step)

    # Define the actions the agent will take in each step
    def step(self):
        ### If the agent is at one of  the exit, mark as exited
        if self.pos in self.model.exit_locations:
            self.found_exit = True  # Set the agent's exit status to True
            self.model.grid.remove_agent(self)  # MESA function to remove the agent from the grid
          #  self.model.schedule.remove(self) ###remove from scheduler
            self.remove()  #self.remove() to remove from AgentSet
            self.model.cumulative_exited += 1  # Count this agent in cumulative exited agents
        else:
            # MESA `get_neighborhood` checks the agent's vision area for the exit
            vision_area = self.model.grid.get_neighborhood(self.pos, moore=True, radius=self.vision, include_center=False)
            exit_in_vision = any(exit_location in vision_area for exit_location in self.model.exit_locations)
            
            # Move towards the exit if it's in sight, otherwise move randomly
            if exit_in_vision:
                self.move_towards_exit()
            else:
                self.move_randomly()

###################
### MODEL CLASS ###
###################

# Define the model class to handle the overall environment
class FloorPlanModel(Model):
    def __init__(self, width, height, num_agents, num_crew, agent_vision): 
        super().__init__()  # `Model` class initialization
        
        # Basic model settings
        self.num_agents = num_agents
        self.num_crew = num_crew
        self.agent_vision = agent_vision
        self.grid = MultiGrid(width, height, False)  # MESA grid with dimensions; False means no wrapping
        self.exit_locations = [(9, 0), (21, 0), (33, 0), (45, 0), (9, 27), (21, 27), (33, 27), (45, 27), (111, 0), (123, 0), (135, 0), (147, 0), (111, 27), (123, 27), (135, 27), (147, 27)]  ### List of multiple exit locations
        self.obstacles = [(4, 9), (4, 10), (4, 11), (4, 12), (4, 13), (4, 14), (4, 15), (4, 16), (4, 17), (4, 18), (5, 9), (5, 10), (5, 11), (5, 12), (5, 13), (5, 14), (5, 15), (5, 16), (5, 17), (5, 18), (6, 9), (6, 10), (6, 11), (6, 12), (6, 13), (6, 14), (6, 15), (6, 16), (6, 17), (6, 18), (7, 9), (7, 10), (7, 11), (7, 12), (7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (8, 9), (8, 10), (8, 11), (8, 12), (8, 13), (8, 14), (8, 15), (8, 16), (8, 17), (8, 18), (9, 9), (9, 10), (9, 11), (9, 12), (9, 13), (9, 14), (9, 15), (9, 16), (9, 17), (9, 18), (12, 7), (12, 8), (12, 9), (12, 10), (12, 11), (12, 12), (12, 13), (12, 14), (12, 15), (12, 16), (12, 17), (12, 18), (12, 19), (12, 20), (13, 7), (13, 8), (13, 9), (13, 10), (13, 11), (13, 12), (13, 13), (13, 14), (13, 15), (13, 16), (13, 17), (13, 18), (13, 19), (13, 20), (14, 7), (14, 8), (14, 9), (14, 10), (14, 11), (14, 12), (14, 13), (14, 14), (14, 15), (14, 16), (14, 17), (14, 18), (14, 19), (14, 20), (15, 7), (15, 8), (15, 9), (15, 10), (15, 11), (15, 12), (15, 13), (15, 14), (15, 15), (15, 16), (15, 17), (15, 18), (15, 19), (15, 20), (16, 7), (16, 8), (16, 9), (16, 10), (16, 11), (16, 12), (16, 13), (16, 14), (16, 15), (16, 16), (16, 17), (16, 18), (16, 19), (16, 20), (17, 7), (17, 8), (17, 9), (17, 10), (17, 11), (17, 12), (17, 13), (17, 14), (17, 15), (17, 16), (17, 17), (17, 18), (17, 19), (17, 20), (18, 7), (18, 8), (18, 9), (18, 10), (18, 11), (18, 12), (18, 13), (18, 14), (18, 15), (18, 16), (18, 17), (18, 18), (18, 19), (18, 20), (19, 7), (19, 8), (19, 9), (19, 10), (19, 11), (19, 12), (19, 13), (19, 14), (19, 15), (19, 16), (19, 17), (19, 18), (19, 19), (19, 20), (20, 7), (20, 8), (20, 9), (20, 10), (20, 11), (20, 12), (20, 13), (20, 14), (20, 15), (20, 16), (20, 17), (20, 18), (20, 19), (20, 20), (21, 7), (21, 8), (21, 9), (21, 10), (21, 11), (21, 12), (21, 13), (21, 14), (21, 15), (21, 16), (21, 17), (21, 18), (21, 19), (21, 20), (22, 7), (22, 8), (22, 9), (22, 10), (22, 11), (22, 12), (22, 13), (22, 14), (22, 15), (22, 16), (22, 17), (22, 18), (22, 19), (22, 20), (23, 7), (23, 8), (23, 9), (23, 10), (23, 11), (23, 12), (23, 13), (23, 14), (23, 15), (23, 16), (23, 17), (23, 18), (23, 19), (23, 20), (24, 7), (24, 8), (24, 9), (24, 10), (24, 11), (24, 12), (24, 13), (24, 14), (24, 15), (24, 16), (24, 17), (24, 18), (24, 19), (24, 20), (25, 7), (25, 8), (25, 9), (25, 10), (25, 11), (25, 12), (25, 13), (25, 14), (25, 15), (25, 16), (25, 17), (25, 18), (25, 19), (25, 20), (26, 7), (26, 8), (26, 9), (26, 10), (26, 11), (26, 12), (26, 13), (26, 14), (26, 15), (26, 16), (26, 17), (26, 18), (26, 19), (26, 20), (27, 7), (27, 8), (27, 9), (27, 10), (27, 11), (27, 12), (27, 13), (27, 14), (27, 15), (27, 16), (27, 17), (27, 18), (27, 19), (27, 20), (28, 7), (28, 8), (28, 9), (28, 10), (28, 11), (28, 12), (28, 13), (28, 14), (28, 15), (28, 16), (28, 17), (28, 18), (28, 19), (28, 20), (29, 7), (29, 8), (29, 9), (29, 10), (29, 11), (29, 12), (29, 13), (29, 14), (29, 15), (29, 16), (29, 17), (29, 18), (29, 19), (29, 20), (30, 7), (30, 8), (30, 9), (30, 10), (30, 11), (30, 12), (30, 13), (30, 14), (30, 15), (30, 16), (30, 17), (30, 18), (30, 19), (30, 20), (31, 7), (31, 8), (31, 9), (31, 10), (31, 11), (31, 12), (31, 13), (31, 14), (31, 15), (31, 16), (31, 17), (31, 18), (31, 19), (31, 20), (32, 7), (32, 8), (32, 9), (32, 10), (32, 11), (32, 12), (32, 13), (32, 14), (32, 15), (32, 16), (32, 17), (32, 18), (32, 19), (32, 20), (33, 7), (33, 8), (33, 9), (33, 10), (33, 11), (33, 12), (33, 13), (33, 14), (33, 15), (33, 16), (33, 17), (33, 18), (33, 19), (33, 20), (34, 7), (34, 8), (34, 9), (34, 10), (34, 11), (34, 12), (34, 13), (34, 14), (34, 15), (34, 16), (34, 17), (34, 18), (34, 19), (34, 20), (35, 7), (35, 8), (35, 9), (35, 10), (35, 11), (35, 12), (35, 13), (35, 14), (35, 15), (35, 16), (35, 17), (35, 18), (35, 19), (35, 20), (36, 7), (36, 8), (36, 9), (36, 10), (36, 11), (36, 12), (36, 13), (36, 14), (36, 15), (36, 16), (36, 17), (36, 18), (36, 19), (36, 20), (37, 7), (37, 8), (37, 9), (37, 10), (37, 11), (37, 12), (37, 13), (37, 14), (37, 15), (37, 16), (37, 17), (37, 18), (37, 19), (37, 20), (38, 7), (38, 8), (38, 9), (38, 10), (38, 11), (38, 12), (38, 13), (38, 14), (38, 15), (38, 16), (38, 17), (38, 18), (38, 19), (38, 20), (39, 7), (39, 8), (39, 9), (39, 10), (39, 11), (39, 12), (39, 13), (39, 14), (39, 15), (39, 16), (39, 17), (39, 18), (39, 19), (39, 20), (44, 11), (44, 12), (44, 13), (44, 14), (44, 15), (44, 16), (45, 11), (45, 12), (45, 13), (45, 14), (45, 15), (45, 16), (46, 11), (46, 12), (46, 13), (46, 14), (46, 15), (46, 16), (47, 11), (47, 12), (47, 13), (47, 14), (47, 15), (47, 16), (48, 9), (48, 10), (48, 11), (48, 12), (48, 13), (48, 14), (48, 15), (48, 16), (48, 17), (48, 18), (49, 9), (49, 10), (49, 11), (49, 12), (49, 13), (49, 14), (49, 15), (49, 16), (49, 17), (49, 18), (50, 9), (50, 10), (50, 11), (50, 12), (50, 13), (50, 14), (50, 15), (50, 16), (50, 17), (50, 18), (51, 9), (51, 10), (51, 11), (51, 12), (51, 13), (51, 14), (51, 15), (51, 16), (51, 17), (51, 18), (52, 9), (52, 10), (52, 11), (52, 12), (52, 13), (52, 14), (52, 15), (52, 16), (52, 17), (52, 18), (53, 9), (53, 10), (53, 11), (53, 12), (53, 13), (53, 14), (53, 15), (53, 16), (53, 17), (53, 18), (56, 11), (56, 12), (56, 13), (56, 14), (56, 15), (56, 16), (57, 11), (57, 12), (57, 13), (57, 14), (57, 15), (57, 16), (58, 11), (58, 12), (58, 13), (58, 14), (58, 15), (58, 16), (59, 11), (59, 12), (59, 13), (59, 14), (59, 15), (59, 16), (60, 11), (60, 12), (60, 13), (60, 14), (60, 15), (60, 16), (61, 11), (61, 12), (61, 13), (61, 14), (61, 15), (61, 16), (62, 11), (62, 12), (62, 13), (62, 14), (62, 15), (62, 16), (63, 11), (63, 12), (63, 13), (63, 14), (63, 15), (63, 16), (64, 7), (64, 8), (64, 9), (64, 10), (64, 11), (64, 12), (64, 13), (64, 14), (64, 15), (64, 16), (64, 17), (64, 18), (64, 19), (64, 20), (65, 7), (65, 8), (65, 9), (65, 10), (65, 11), (65, 12), (65, 13), (65, 14), (65, 15), (65, 16), (65, 17), (65, 18), (65, 19), (65, 20), (66, 7), (66, 8), (66, 9), (66, 10), (66, 11), (66, 12), (66, 13), (66, 14), (66, 15), (66, 16), (66, 17), (66, 18), (66, 19), (66, 20), (67, 7), (67, 8), (67, 9), (67, 10), (67, 11), (67, 12), (67, 13), (67, 14), (67, 15), (67, 16), (67, 17), (67, 18), (67, 19), (67, 20), (68, 7), (68, 8), (68, 9), (68, 10), (68, 11), (68, 12), (68, 13), (68, 14), (68, 15), (68, 16), (68, 17), (68, 18), (68, 19), (68, 20), (69, 7), (69, 8), (69, 9), (69, 10), (69, 11), (69, 12), (69, 13), (69, 14), (69, 15), (69, 16), (69, 17), (69, 18), (69, 19), (69, 20), (70, 7), (70, 8), (70, 9), (70, 10), (70, 11), (70, 12), (70, 13), (70, 14), (70, 15), (70, 16), (70, 17), (70, 18), (70, 19), (70, 20), (71, 7), (71, 8), (71, 9), (71, 10), (71, 11), (71, 12), (71, 13), (71, 14), (71, 15), (71, 16), (71, 17), (71, 18), (71, 19), (71, 20), (72, 7), (72, 8), (72, 9), (72, 10), (72, 11), (72, 12), (72, 13), (72, 14), (72, 15), (72, 16), (72, 17), (72, 18), (72, 19), (72, 20), (73, 7), (73, 8), (73, 9), (73, 10), (73, 11), (73, 12), (73, 13), (73, 14), (73, 15), (73, 16), (73, 17), (73, 18), (73, 19), (73, 20), (74, 7), (74, 8), (74, 9), (74, 10), (74, 11), (74, 12), (74, 13), (74, 14), (74, 15), (74, 16), (74, 17), (74, 18), (74, 19), (74, 20), (75, 7), (75, 8), (75, 9), (75, 10), (75, 11), (75, 12), (75, 13), (75, 14), (75, 15), (75, 16), (75, 17), (75, 18), (75, 19), (75, 20), (76, 7), (76, 8), (76, 9), (76, 10), (76, 11), (76, 12), (76, 13), (76, 14), (76, 15), (76, 16), (76, 17), (76, 18), (76, 19), (76, 20), (77, 7), (77, 8), (77, 9), (77, 10), (77, 11), (77, 12), (77, 13), (77, 14), (77, 15), (77, 16), (77, 17), (77, 18), (77, 19), (77, 20), (78, 7), (78, 8), (78, 9), (78, 10), (78, 11), (78, 12), (78, 13), (78, 14), (78, 15), (78, 16), (78, 17), (78, 18), (78, 19), (78, 20), (79, 7), (79, 8), (79, 9), (79, 10), (79, 11), (79, 12), (79, 13), (79, 14), (79, 15), (79, 16), (79, 17), (79, 18), (79, 19), (79, 20), (80, 7), (80, 8), (80, 9), (80, 10), (80, 11), (80, 12), (80, 13), (80, 14), (80, 15), (80, 16), (80, 17), (80, 18), (80, 19), (80, 20), (81, 7), (81, 8), (81, 9), (81, 10), (81, 11), (81, 12), (81, 13), (81, 14), (81, 15), (81, 16), (81, 17), (81, 18), (81, 19), (81, 20), (82, 7), (82, 8), (82, 9), (82, 10), (82, 11), (82, 12), (82, 13), (82, 14), (82, 15), (82, 16), (82, 17), (82, 18), (82, 19), (82, 20), (83, 7), (83, 8), (83, 9), (83, 10), (83, 11), (83, 12), (83, 13), (83, 14), (83, 15), (83, 16), (83, 17), (83, 18), (83, 19), (83, 20), (84, 11), (84, 12), (84, 13), (84, 14), (84, 15), (84, 16), (84, 17), (84, 18), (84, 19), (84, 20), (85, 11), (85, 12), (85, 13), (85, 14), (85, 15), (85, 16), (85, 17), (85, 18), (85, 19), (85, 20), (86, 11), (86, 12), (86, 13), (86, 14), (86, 15), (86, 16), (86, 17), (86, 18), (86, 19), (86, 20), (87, 11), (87, 12), (87, 13), (87, 14), (87, 15), (87, 16), (87, 17), (87, 18), (87, 19), (87, 20), (88, 11), (88, 12), (88, 13), (88, 14), (88, 15), (88, 16), (88, 17), (88, 18), (88, 19), (88, 20), (89, 11), (89, 12), (89, 13), (89, 14), (89, 15), (89, 16), (89, 17), (89, 18), (89, 19), (89, 20), (90, 11), (90, 12), (90, 13), (90, 14), (90, 15), (90, 16), (90, 17), (90, 18), (90, 19), (90, 20), (91, 11), (91, 12), (91, 13), (91, 14), (91, 15), (91, 16), (91, 17), (91, 18), (91, 19), (91, 20), (92, 7), (92, 8), (92, 9), (92, 10), (92, 11), (92, 12), (92, 13), (92, 14), (92, 15), (92, 16), (92, 17), (92, 18), (92, 19), (92, 20), (93, 7), (93, 8), (93, 9), (93, 10), (93, 11), (93, 12), (93, 13), (93, 14), (93, 15), (93, 16), (93, 17), (93, 18), (93, 19), (93, 20), (94, 7), (94, 8), (94, 9), (94, 10), (94, 11), (94, 12), (94, 13), (94, 14), (94, 15), (94, 16), (94, 17), (94, 18), (94, 19), (94, 20), (95, 7), (95, 8), (95, 9), (95, 10), (95, 11), (95, 12), (95, 13), (95, 14), (95, 15), (95, 16), (95, 17), (95, 18), (95, 19), (95, 20), (96, 7), (96, 8), (96, 9), (96, 10), (96, 11), (96, 12), (96, 13), (96, 14), (96, 15), (96, 16), (96, 17), (96, 18), (96, 19), (96, 20), (97, 7), (97, 8), (97, 9), (97, 10), (97, 11), (97, 12), (97, 13), (97, 14), (97, 15), (97, 16), (97, 17), (97, 18), (97, 19), (97, 20), (98, 7), (98, 8), (98, 9), (98, 10), (98, 11), (98, 12), (98, 13), (98, 14), (98, 15), (98, 16), (98, 17), (98, 18), (98, 19), (98, 20), (99, 7), (99, 8), (99, 9), (99, 10), (99, 11), (99, 12), (99, 13), (99, 14), (99, 15), (99, 16), (99, 17), (99, 18), (99, 19), (99, 20), (100, 7), (100, 8), (100, 9), (100, 10), (100, 11), (100, 12), (100, 13), (100, 14), (101, 7), (101, 8), (101, 9), (101, 10), (101, 11), (101, 12), (101, 13), (101, 14), (102, 7), (102, 8), (102, 9), (102, 10), (102, 11), (102, 12), (102, 13), (102, 14), (103, 7), (103, 8), (103, 9), (103, 10), (103, 11), (103, 12), (103, 13), (103, 14), (104, 7), (104, 8), (104, 9), (104, 10), (104, 11), (104, 12), (104, 13), (104, 14), (105, 7), (105, 8), (105, 9), (105, 10), (105, 11), (105, 12), (105, 13), (105, 14), (106, 7), (106, 8), (106, 9), (106, 10), (106, 11), (106, 12), (106, 13), (106, 14), (107, 7), (107, 8), (107, 9), (107, 10), (107, 11), (107, 12), (107, 13), (107, 14), (108, 7), (108, 8), (108, 9), (108, 10), (108, 11), (108, 12), (108, 13), (108, 14), (108, 15), (108, 16), (108, 17), (108, 18), (109, 7), (109, 8), (109, 9), (109, 10), (109, 11), (109, 12), (109, 13), (109, 14), (109, 15), (109, 16), (109, 17), (109, 18), (110, 7), (110, 8), (110, 9), (110, 10), (110, 11), (110, 12), (110, 13), (110, 14), (110, 15), (110, 16), (110, 17), (110, 18), (111, 7), (111, 8), (111, 9), (111, 10), (111, 11), (111, 12), (111, 13), (111, 14), (111, 15), (111, 16), (111, 17), (111, 18), (112, 7), (112, 8), (112, 9), (112, 10), (112, 11), (112, 12), (112, 13), (112, 14), (112, 15), (112, 16), (112, 17), (112, 18), (113, 7), (113, 8), (113, 9), (113, 10), (113, 11), (113, 12), (113, 13), (113, 14), (113, 15), (113, 16), (113, 17), (113, 18), (114, 7), (114, 8), (114, 9), (114, 10), (114, 11), (114, 12), (114, 13), (114, 14), (114, 15), (114, 16), (114, 17), (114, 18), (115, 7), (115, 8), (115, 9), (115, 10), (115, 11), (115, 12), (115, 13), (115, 14), (115, 15), (115, 16), (115, 17), (115, 18), (116, 7), (116, 8), (116, 9), (116, 10), (116, 11), (116, 12), (116, 13), (116, 14), (116, 15), (116, 16), (116, 17), (116, 18), (117, 7), (117, 8), (117, 9), (117, 10), (117, 11), (117, 12), (117, 13), (117, 14), (117, 15), (117, 16), (117, 17), (117, 18), (118, 10), (118, 11), (118, 12), (118, 13), (118, 14), (118, 15), (118, 16), (118, 17), (119, 10), (119, 11), (119, 12), (119, 13), (119, 14), (119, 15), (119, 16), (119, 17), (120, 10), (120, 11), (120, 12), (120, 13), (120, 14), (120, 15), (120, 16), (120, 17), (121, 10), (121, 11), (121, 12), (121, 13), (121, 14), (121, 15), (121, 16), (121, 17), (122, 10), (122, 11), (122, 12), (122, 13), (122, 14), (122, 15), (122, 16), (122, 17), (123, 10), (123, 11), (123, 12), (123, 13), (123, 14), (123, 15), (123, 16), (123, 17), (124, 10), (124, 11), (124, 12), (124, 13), (124, 14), (124, 15), (124, 16), (124, 17), (125, 10), (125, 11), (125, 12), (125, 13), (125, 14), (125, 15), (125, 16), (125, 17), (126, 10), (126, 11), (126, 12), (126, 13), (126, 14), (126, 15), (126, 16), (126, 17), (127, 10), (127, 11), (127, 12), (127, 13), (127, 14), (127, 15), (127, 16), (127, 17), (128, 10), (128, 11), (128, 12), (128, 13), (128, 14), (128, 15), (128, 16), (128, 17), (129, 10), (129, 11), (129, 12), (129, 13), (129, 14), (129, 15), (129, 16), (129, 17), (130, 10), (130, 11), (130, 12), (130, 13), (130, 14), (130, 15), (130, 16), (130, 17), (131, 10), (131, 11), (131, 12), (131, 13), (131, 14), (131, 15), (131, 16), (131, 17), (132, 10), (132, 11), (132, 12), (132, 13), (132, 14), (132, 15), (132, 16), (132, 17), (133, 10), (133, 11), (133, 12), (133, 13), (133, 14), (133, 15), (133, 16), (133, 17), (134, 10), (134, 11), (134, 12), (134, 13), (134, 14), (134, 15), (134, 16), (134, 17), (135, 10), (135, 11), (135, 12), (135, 13), (135, 14), (135, 15), (135, 16), (135, 17), (136, 10), (136, 11), (136, 12), (136, 13), (136, 14), (136, 15), (136, 16), (136, 17), (137, 10), (137, 11), (137, 12), (137, 13), (137, 14), (137, 15), (137, 16), (137, 17), (138, 10), (138, 11), (138, 12), (138, 13), (138, 14), (138, 15), (138, 16), (138, 17), (139, 10), (139, 11), (139, 12), (139, 13), (139, 14), (139, 15), (139, 16), (139, 17), (140, 10), (140, 11), (140, 12), (140, 13), (140, 14), (140, 15), (140, 16), (140, 17), (141, 10), (141, 11), (141, 12), (141, 13), (141, 14), (141, 15), (141, 16), (141, 17), (142, 10), (142, 11), (142, 12), (142, 13), (142, 14), (142, 15), (142, 16), (142, 17), (143, 10), (143, 11), (143, 12), (143, 13), (143, 14), (143, 15), (143, 16), (143, 17), (144, 10), (144, 11), (144, 12), (144, 13), (144, 14), (144, 15), (144, 16), (144, 17), (145, 10), (145, 11), (145, 12), (145, 13), (145, 14), (145, 15), (145, 16), (145, 17), (146, 10), (146, 11), (146, 12), (146, 13), (146, 14), (146, 15), (146, 16), (146, 17), (147, 10), (147, 11), (147, 12), (147, 13), (147, 14), (147, 15), (147, 16), (147, 17), (148, 10), (148, 11), (148, 12), (148, 13), (148, 14), (148, 15), (148, 16), (148, 17), (149, 10), (149, 11), (149, 12), (149, 13), (149, 14), (149, 15), (149, 16), (149, 17), (150, 10), (150, 11), (150, 12), (150, 13), (150, 14), (150, 15), (150, 16), (150, 17), (151, 10), (151, 11), (151, 12), (151, 13), (151, 14), (151, 15), (151, 16), (151, 17), (152, 11), (152, 12), (152, 13), (152, 14), (152, 15), (152, 16), (153, 11), (153, 12), (153, 13), (153, 14), (153, 15), (153, 16), (154, 11), (154, 12), (154, 13), (154, 14), (154, 15), (154, 16), (155, 11), (155, 12), (155, 13), (155, 14), (155, 15), (155, 16),
                         (0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3),
                         (2, 0), (2, 1), (3, 0), (3, 1),
                         (0, 24), (0, 25), (0, 26), (0, 27), (1, 24), (1, 25), (1, 26), (1, 27),
                         (2, 26), (2, 27), (3, 26), (3, 27),
                         (156, 0), (157, 0), (156, 1), (157, 1),
                         (158, 0), (158, 1), (158, 2), (158, 3), (159, 0), (159, 1), (159, 2), (159, 3),
                         (156, 26), (156, 27), (157, 26), (157, 27),
                         (158, 24), (158, 25), (158, 26), (158, 27), (159, 24), (159, 25), (159, 26), (159, 27)]
        self.signs = [(15, 15), (20, 20), (25, 25)]
         
        self.cumulative_exited = 0 # Initialize cumulative exited count
        self.agentss = [] #make list with amount of agents

        # Initialize DataCollector (MESA tool for tracking metrics across steps)
        self.datacollector = DataCollector(
            model_reporters={
                "Active Agents": lambda m: len(m.agents),  # Count of agents still active
                "Exited Agents": lambda m: sum(1 for agent in m.agents if isinstance(agent, NavigationAgent) and agent.found_exit),
                "Cumulative Exited Agents": lambda m: m.cumulative_exited,  # Cumulative exited count
                "Agents per Cell": self.count_agents_per_cell  # Counts agents in each cell
            },
            agent_reporters={
                "Found Exit": lambda a: a.found_exit if isinstance(a, NavigationAgent) else None  # Reports exit status per agent
            }
        )

        self.place_agents(agent_vision)  # Place agents on the grid
        self.datacollector.collect(self) # Collect data at the start of the simulation
        

    ### Function to place agents on particular entry points, these points will be the doors to top deck and the stairs
    def place_agents(self, agent_vision):
        #### Define the fixed entries (spawn points) of crew and others
        crewrooms = [(122,9),(122,18), (134,9),(134,18), (148,9),(148,18)]  #width and height!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        num_crewrooms = len(crewrooms)
        entries = [(7, 8), (7, 19), (109, 19), (109, 6)] 
        num_entries = len(entries)
    
        for i in range(self.num_agents):
            agent = NavigationAgent(self, vision=agent_vision)  # Create agent with the specified vision ###changed self to i why, idk?
            spawn_agent = entries[i % num_entries]  # Alternate between the spawn points
            cell_contents = self.grid.get_cell_list_contents(spawn_agent)
            if len(cell_contents) < 8:
                self.grid.place_agent(agent, spawn_agent)  # Place the agent at the spawn point, only with <8 agents incell
                self.agentss.append(agent) #create passenger agent (blue)

        for i in range(self.num_crew):
            crew = CrewMember(self, vision=agent_vision)  #its not referring to i but to 'self' so idk if thats the way
            spawn_crew = crewrooms[i % num_crewrooms]
            cell_contents = self.grid.get_cell_list_contents(spawn_crew)
            if len(cell_contents) < 8:
                self.grid.place_agent(crew, spawn_crew)  # Place the agent at the spawn point, only with <8 agents incell
                self.agentss.append(crew) #create crew agent (red)

    # Function to count agents in each cell
    def count_agents_per_cell(self):
        agent_counts = {}  # Dictionary to store agent counts by cell position
        # MESA `coord_iter` function iterates over grid cells and their contents
        for cell in self.grid.coord_iter():
            cell_contents, (x, y) = cell  # Unpack cell contents and coordinates
            # Count NavigationAgents in each cell
            nav_agent_count = sum(1 for obj in cell_contents if isinstance(obj, NavigationAgent))
            if nav_agent_count > 0:
                agent_counts[(x, y)] = nav_agent_count
        return agent_counts
    
    # Function to get the grid data for visualization
    def get_grid(self):
        # 0: empty, 1: obstacle, 2: agent, 3: exit, 4: sign
        grid_data = np.zeros((self.grid.height, self.grid.width))
        
        # Mark obstacles on the grid
        for x, y in self.obstacles:
            grid_data[y, x] = 1
        
        # Mark agents on the grid
        for agent in self.agents:
            if isinstance(agent, NavigationAgent):
                x, y = agent.pos
                grid_data[y, x] = 2
        
        #### Mark signs and exits, multiple
        for x, y in self.signs:
            grid_data[y, x] = 4
        for exit_x,exit_y in self.exit_locations:
            grid_data[exit_y,exit_x]=16      #3 exit locations, if there are more change!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
       # exit_x, exit_y = self.exit_locations
        grid_data[exit_y, exit_x] = 16
        return grid_data

    # Model step function to update the simulation
    def step(self):
        self.agents.do("step")  # MESA 3.0 function to execute the `step` function of each agent
        self.datacollector.collect(self)  # MESA DataCollector collects metrics at each step

    

In [6]:
# Run the model and see the results
model = FloorPlanModel(160, 28, 2, 36, 5) # run the model with 30x30 grid and 100 agents and a vision of 5 because more agents its trippinnnn
for i in range(100): # run the model for 100 steps
    model.step() # step the model by 1

# Collect the data from the model
model_data = model.datacollector.get_model_vars_dataframe()
agent_data = model.datacollector.get_agent_vars_dataframe()


In [7]:
model_data # print the model data

,Active Agents,Exited Agents,Cumulative Exited Agents,Agents per Cell
0,38,0,0,"{(7, 8): 1, (7, 19): 1}"
1,38,0,0,"{(6, 7): 1, (6, 20): 1}"
2,38,0,0,"{(5, 8): 1, (7, 19): 1}"
3,38,0,0,"{(4, 8): 1, (8, 20): 1}"
4,38,0,0,"{(3, 7): 1, (8, 19): 1}"
...,...,...,...,...
96,36,0,2,{}
97,36,0,2,{}
98,36,0,2,{}
99,36,0,2,{}


In [8]:
agent_data # print the agent data

Found Exit
Step AgentID           
0    1            False
     2            False
     3             None
     4             None
     5             None
...                 ...
100  34            None
     35            None
     36            None
     37            None
     38            None

[3664 rows x 1 columns]

# Visualization with User Interface
You don't have to understand every single line in the visualisation code below. Please understand the code to extend that you can introduce changes to it when needed

In [9]:
# Visualization Function
def plot_grid(model, ax):
    rc("animation", embed_limit=700)  # Set a higher limit in MB to allow smoother animation playback
    grid_data = model.get_grid()  # Retrieve the current state of the grid from the model
    ax.clear()  # Clear any previous plots on the axes to prevent overlap in visualizations
    
    # Define color mappings:
    # 0 (empty) -> white, 1 (obstacle) -> black, 2 (agent) -> blue,
    # 3 (exit) -> green, 4 (sign) -> orange
    cmap = mcolors.ListedColormap(['white', 'black', 'blue', 'green', 'orange'])
    bounds = [0, 1, 2, 3, 4]  # Boundaries to separate each category
    norm = mcolors.BoundaryNorm(bounds, cmap.N)  # Normalizes values to assign colors to each category
    
    # Display the grid data with color mapping applied.
    # Setting 'origin' to 'lower' places the (0,0) coordinate at the bottom-left.
    ax.imshow(grid_data, cmap=cmap, norm=norm, origin='lower')
    
    # Add grid lines for better cell visibility
    ax.grid(which='both', color='gray', linestyle='-', linewidth=2)
    
    # Customize grid display: set grid to start from -0.5 with labels at intervals of 1
    # Set minor ticks for cell boundaries, with thicker and darker lines
    ax.set_xticks(np.arange(-0.5, model.grid.width, 1), minor=True)
    ax.set_yticks(np.arange(-0.5, model.grid.height, 1), minor=True)
    ax.grid(which='minor', color='gray', linestyle='-', linewidth=1.5)  # Thicker, darker lines for cell boundaries

    # Set major ticks for labels at intervals of 5, with lighter and thinner lines
    ax.set_xticks(np.arange(0, model.grid.width, 5), minor=False)
    ax.set_yticks(np.arange(0, model.grid.height, 5), minor=False)
    ax.grid(which='major', color='lightgray', linestyle='-', linewidth=0)  # invisible line
    
    ### Label the exit point on the grid in red text for easy identification
    for exit_x, exit_y in model.exit_locations:
        ax.text(exit_x, exit_y, 'EXIT', ha='center', va='center', fontsize=12, color='red', fontweight='bold')
    # Add labels for each sign location in black text for visibility
    for sign_x,sign_y in model.signs:
        ax.text(sign_x, sign_y, 'SIGN', ha='center', va='center', fontsize=10, color='black', fontweight='bold')

    # Set the title to show the current step number in the model
    ax.set_title(f"Step {model.steps}", fontsize=16)
    ax.tick_params(axis='both', which='major', labelsize=7)  # Control tick label size
    ####
    # Draw movement arrows for agents to show the direction they are traveling
    for agent in model.agents:
        if isinstance(agent, NavigationAgent):
            ax.scatter(agent.pos[0], agent.pos[1], color='blue', s=40)  # Blue for passenger agents
        elif isinstance(agent, CrewMember):
            ax.scatter(agent.pos[0], agent.pos[1], color='red', s=40)  # Red for crew agents

    # Draw movement arrows for agents to show the direction they are traveling
    for agent in model.agents:
        if isinstance(agent, NavigationAgent) and agent.previous_pos:
            start_x, start_y = agent.previous_pos  # Previous position of the agent
            end_x, end_y = agent.pos  # Current position of the agent
            
            # Draw an arrow from the previous position to the current position
            ax.arrow(
                start_x, start_y,
                end_x - start_x, end_y - start_y,
                head_width=0.3, head_length=0.3, fc='yellow', ec='yellow'  # Yellow arrow for movement direction
            )
    ####        

# Animation Update Function
def update(frame, model, ax):
    # For every frame, update the model state if it's not the first frame
    if frame > 0:
        model.step()  # Run one step of the model simulation
    plot_grid(model, ax)  # Redraw the grid with updated agent positions
    
# Run the Animation with a larger figure size
def run_animation(model, steps):
    fig, ax = plt.subplots(figsize=(15, 5))  # Create a 10x10 figure for the plot!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    plot_grid(model, ax)  # Plot the initial grid state
    # Create an animation that updates the grid for each step
    anim = FuncAnimation(fig, update, frames=steps+1, fargs=(model, ax), repeat=False)
    plt.close(fig)  # Close the figure after animation creation to avoid duplicate displays
    return anim

# Your imports and function definitions remain the same, up until where you initialize and run the model
import ipywidgets as widgets  # For interactive widgets (slider, button)
from IPython.display import display, HTML  # To display widgets and HTML animations
import time  # For tracking elapsed time

# Slider to choose the number of agents
agent_slider = widgets.IntSlider(
    value=10,      # Default starting number of agents
    min=10,        # Minimum number of agents allowed
    max=500,       # Maximum number of agents allowed
    step=10,       # Step size for slider increments
    description='Num Agents:',  # Label for the slider
    continuous_update=False     # Only update value when slider is released
)

# Slider to choose the number of time steps (frames) for the animation
time_step_slider = widgets.IntSlider(
    value=50,      # Default starting number of steps
    min=1,         # Minimum time steps allowed
    max=500,       # Maximum time steps allowed
    step=1,        # Step size for slider increments
    description='Time Steps:',  # Label for the slider
    continuous_update=False     # Only update value when slider is released
)

# Slider to choose the vision range (vision radius) of agents
vision_slider = widgets.IntSlider(
    value=5,      # Default starting vision range
    min=1,        # Minimum vision range
    max=20,       # Maximum vision range
    step=1,       # Step size for slider increments
    description='Vision:',  # Label for the slider
    continuous_update=False     # Only update value when slider is released
)

# Button to start the simulation with current slider settings
run_button = widgets.Button(description="Run Simulation")

# Output widget for displaying the animation and elapsed time
output_widget = widgets.Output()

# Label to show elapsed time during the simulation
elapsed_time_label = widgets.Label(value="Elapsed time: 0.0 seconds")

# Flag variable to control timing function
stop_timer = False

# Display the interface with sliders, button, label, and output area
display(agent_slider, time_step_slider, vision_slider, run_button, elapsed_time_label, output_widget)

# Define the function to initialize and run the model
def run_model(change):
    global stop_timer, model_data, agent_data  # Allow variables to be accessed globally
    with output_widget:  # Use output widget to display the animation
        output_widget.clear_output()  # Clear any previous output
        num_agents = agent_slider.value  # Get the number of agents from the slider
        time_steps = time_step_slider.value  # Get the number of steps from the slider
        agent_vision = vision_slider.value  # Get the vision range from the slider
        model = FloorPlanModel(width=160, height=28, num_agents=num_agents, num_crew=2, agent_vision=agent_vision)  # Initialize the model!!!!!!!!!!!


        # Reset timer flag and start timing for the animation
        stop_timer = False
        start_time = time.time()

        def update_time_label():
            while not stop_timer:  # Keep updating time label until timer stops
                elapsed_time = time.time() - start_time
                elapsed_time_label.value = f"Elapsed time: {elapsed_time:.1f} seconds"
                time.sleep(0.1)  # Update every 0.1 seconds for real-time effect

        # Start elapsed time tracking in a separate thread
        import threading
        timer_thread = threading.Thread(target=update_time_label, daemon=True)
        timer_thread.start()

        # Run the animation with selected number of steps
        anim = run_animation(model, steps=time_steps)

        # Display the animation output in HTML format
        output = HTML(anim.to_jshtml())
        display(output)

        # Stop the timer after the simulation completes
        stop_timer = True
        elapsed_time = time.time() - start_time
        elapsed_time_label.value = f"Total elapsed time: {elapsed_time:.1f} seconds"

        # Retrieve and display model and agent data after simulation completes
        model_data = model.datacollector.get_model_vars_dataframe()  # Data for the model over time
        agent_data = model.datacollector.get_agent_vars_dataframe()  # Data for each agent over time
    return model, model_data, agent_data  # Return model and data for further inspection

# Attach the run_model function to the run button click event
run_button.on_click(run_model)




IntSlider(value=10, continuous_update=False, description='Num Agents:', max=500, min=10, step=10)

IntSlider(value=50, continuous_update=False, description='Time Steps:', max=500, min=1)

IntSlider(value=5, continuous_update=False, description='Vision:', max=20, min=1)

Button(description='Run Simulation', style=ButtonStyle())

Label(value='Elapsed time: 0.0 seconds')

Output()

In [10]:
print("\nModel Data:")
model_data


Model Data:


,Active Agents,Exited Agents,Cumulative Exited Agents,Agents per Cell
0,38,0,0,"{(7, 8): 1, (7, 19): 1}"
1,38,0,0,"{(6, 7): 1, (6, 20): 1}"
2,38,0,0,"{(5, 8): 1, (7, 19): 1}"
3,38,0,0,"{(4, 8): 1, (8, 20): 1}"
4,38,0,0,"{(3, 7): 1, (8, 19): 1}"
...,...,...,...,...
96,36,0,2,{}
97,36,0,2,{}
98,36,0,2,{}
99,36,0,2,{}


In [11]:
print("\nAgent Data:")
agent_data.head(400)



Agent Data:


Found Exit
Step AgentID           
0    1            False
     2            False
     3             None
     4             None
     5             None
...                 ...
10   16            None
     17            None
     18            None
     19            None
     20            None

[400 rows x 1 columns]

In [12]:
#  Uncomment the code below to display the 'Agents per Cell' data for each step

#print("Agents per Cell at each step:")
#for step, agents_per_cell in model_data["Agents per Cell"].items():
#    print(f"\nStep {step}:")
#    for cell, count in agents_per_cell.items():
#        print(f"  Cell {cell}: {count} agent(s)")

In [13]:
coordinates = []
#second number should be end+1
for a in range(158, 160):  
    for b in range(24, 28):  
        coordinates.append((a, b))

print(coordinates)


[(158, 24), (158, 25), (158, 26), (158, 27), (159, 24), (159, 25), (159, 26), (159, 27)]


In [14]:
#coordinates1 = [(5, 10), (5, 11), (5, 12), (5, 13), (5, 14), (5, 15), (5, 16), (5, 17), (5, 18), (5, 19), (6, 10), (6, 11), (6, 12), (6, 13), (6, 14), (6, 15), (6, 16), (6, 17), (6, 18), (6, 19), (7, 10), (7, 11), (7, 12), (7, 13), (7, 14), (7, 15), (7, 16), (7, 17), (7, 18), (7, 19), (8, 10), (8, 11), (8, 12), (8, 13), (8, 14), (8, 15), (8, 16), (8, 17), (8, 18), (8, 19), (9, 10), (9, 11), (9, 12), (9, 13), (9, 14), (9, 15), (9, 16), (9, 17), (9, 18), (9, 19), (10, 10), (10, 11), (10, 12), (10, 13), (10, 14), (10, 15), (10, 16), (10, 17), (10, 18), (10, 19),
                         (13, 8), (13, 9), (13, 10), (13, 11), (13, 12), (13, 13), (13, 14), (13, 15), (13, 16), (13, 17), (13, 18), (13, 19), (13, 20), (13, 21), (14, 8), (14, 9), (14, 10), (14, 11), (14, 12), (14, 13), (14, 14), (14, 15), (14, 16), (14, 17), (14, 18), (14, 19), (14, 20), (14, 21), (15, 8), (15, 9), (15, 10), (15, 11), (15, 12), (15, 13), (15, 14), (15, 15), (15, 16), (15, 17), (15, 18), (15, 19), (15, 20), (15, 21), (16, 8), (16, 9), (16, 10), (16, 11), (16, 12), (16, 13), (16, 14), (16, 15), (16, 16), (16, 17), (16, 18), (16, 19), (16, 20), (16, 21), (17, 8), (17, 9), (17, 10), (17, 11), (17, 12), (17, 13), (17, 14), (17, 15), (17, 16), (17, 17), (17, 18), (17, 19), (17, 20), (17, 21), (18, 8), (18, 9), (18, 10), (18, 11), (18, 12), (18, 13), (18, 14), (18, 15), (18, 16), (18, 17), (18, 18), (18, 19), (18, 20), (18, 21), (19, 8), (19, 9), (19, 10), (19, 11), (19, 12), (19, 13), (19, 14), (19, 15), (19, 16), (19, 17), (19, 18), (19, 19), (19, 20), (19, 21), (20, 8), (20, 9), (20, 10), (20, 11), (20, 12), (20, 13), (20, 14), (20, 15), (20, 16), (20, 17), (20, 18), (20, 19), (20, 20), (20, 21), (21, 8), (21, 9), (21, 10), (21, 11), (21, 12), (21, 13), (21, 14), (21, 15), (21, 16), (21, 17), (21, 18), (21, 19), (21, 20), (21, 21), (22, 8), (22, 9), (22, 10), (22, 11), (22, 12), (22, 13), (22, 14), (22, 15), (22, 16), (22, 17), (22, 18), (22, 19), (22, 20), (22, 21), (23, 8), (23, 9), (23, 10), (23, 11), (23, 12), (23, 13), (23, 14), (23, 15), (23, 16), (23, 17), (23, 18), (23, 19), (23, 20), (23, 21), (24, 8), (24, 9), (24, 10), (24, 11), (24, 12), (24, 13), (24, 14), (24, 15), (24, 16), (24, 17), (24, 18), (24, 19), (24, 20), (24, 21), (25, 8), (25, 9), (25, 10), (25, 11), (25, 12), (25, 13), (25, 14), (25, 15), (25, 16), (25, 17), (25, 18), (25, 19), (25, 20), (25, 21), (26, 8), (26, 9), (26, 10), (26, 11), (26, 12), (26, 13), (26, 14), (26, 15), (26, 16), (26, 17), (26, 18), (26, 19), (26, 20), (26, 21), (27, 8), (27, 9), (27, 10), (27, 11), (27, 12), (27, 13), (27, 14), (27, 15), (27, 16), (27, 17), (27, 18), (27, 19), (27, 20), (27, 21), (28, 8), (28, 9), (28, 10), (28, 11), (28, 12), (28, 13), (28, 14), (28, 15), (28, 16), (28, 17), (28, 18), (28, 19), (28, 20), (28, 21), (29, 8), (29, 9), (29, 10), (29, 11), (29, 12), (29, 13), (29, 14), (29, 15), (29, 16), (29, 17), (29, 18), (29, 19), (29, 20), (29, 21), (30, 8), (30, 9), (30, 10), (30, 11), (30, 12), (30, 13), (30, 14), (30, 15), (30, 16), (30, 17), (30, 18), (30, 19), (30, 20), (30, 21), (31, 8), (31, 9), (31, 10), (31, 11), (31, 12), (31, 13), (31, 14), (31, 15), (31, 16), (31, 17), (31, 18), (31, 19), (31, 20), (31, 21), (32, 8), (32, 9), (32, 10), (32, 11), (32, 12), (32, 13), (32, 14), (32, 15), (32, 16), (32, 17), (32, 18), (32, 19), (32, 20), (32, 21), (33, 8), (33, 9), (33, 10), (33, 11), (33, 12), (33, 13), (33, 14), (33, 15), (33, 16), (33, 17), (33, 18), (33, 19), (33, 20), (33, 21), (34, 8), (34, 9), (34, 10), (34, 11), (34, 12), (34, 13), (34, 14), (34, 15), (34, 16), (34, 17), (34, 18), (34, 19), (34, 20), (34, 21), (35, 8), (35, 9), (35, 10), (35, 11), (35, 12), (35, 13), (35, 14), (35, 15), (35, 16), (35, 17), (35, 18), (35, 19), (35, 20), (35, 21), (36, 8), (36, 9), (36, 10), (36, 11), (36, 12), (36, 13), (36, 14), (36, 15), (36, 16), (36, 17), (36, 18), (36, 19), (36, 20), (36, 21), (37, 8), (37, 9), (37, 10), (37, 11), (37, 12), (37, 13), (37, 14), (37, 15), (37, 16), (37, 17), (37, 18), (37, 19), (37, 20), (37, 21), (38, 8), (38, 9), (38, 10), (38, 11), (38, 12), (38, 13), (38, 14), (38, 15), (38, 16), (38, 17), (38, 18), (38, 19), (38, 20), (38, 21), (39, 8), (39, 9), (39, 10), (39, 11), (39, 12), (39, 13), (39, 14), (39, 15), (39, 16), (39, 17), (39, 18), (39, 19), (39, 20), (39, 21), (40, 8), (40, 9), (40, 10), (40, 11), (40, 12), (40, 13), (40, 14), (40, 15), (40, 16), (40, 17), (40, 18), (40, 19), (40, 20), (40, 21),
                         (45, 12), (45, 13), (45, 14), (45, 15), (45, 16), (45, 17), (46, 12), (46, 13), (46, 14), (46, 15), (46, 16), (46, 17), (47, 12), (47, 13), (47, 14), (47, 15), (47, 16), (47, 17), (48, 12), (48, 13), (48, 14), (48, 15), (48, 16), (48, 17),
                         (49, 10), (49, 11), (49, 12), (49, 13), (49, 14), (49, 15), (49, 16), (49, 17), (49, 18), (49, 19), (50, 10), (50, 11), (50, 12), (50, 13), (50, 14), (50, 15), (50, 16), (50, 17), (50, 18), (50, 19), (51, 10), (51, 11), (51, 12), (51, 13), (51, 14), (51, 15), (51, 16), (51, 17), (51, 18), (51, 19), (52, 10), (52, 11), (52, 12), (52, 13), (52, 14), (52, 15), (52, 16), (52, 17), (52, 18), (52, 19), (53, 10), (53, 11), (53, 12), (53, 13), (53, 14), (53, 15), (53, 16), (53, 17), (53, 18), (53, 19), (54, 10), (54, 11), (54, 12), (54, 13), (54, 14), (54, 15), (54, 16), (54, 17), (54, 18), (54, 19),
                         (57, 12), (57, 13), (57, 14), (57, 15), (57, 16), (57, 17), (58, 12), (58, 13), (58, 14), (58, 15), (58, 16), (58, 17), (59, 12), (59, 13), (59, 14), (59, 15), (59, 16), (59, 17), (60, 12), (60, 13), (60, 14), (60, 15), (60, 16), (60, 17), (61, 12), (61, 13), (61, 14), (61, 15), (61, 16), (61, 17), (62, 12), (62, 13), (62, 14), (62, 15), (62, 16), (62, 17), (63, 12), (63, 13), (63, 14), (63, 15), (63, 16), (63, 17), (64, 12), (64, 13), (64, 14), (64, 15), (64, 16), (64, 17),
                         (65, 8), (65, 9), (65, 10), (65, 11), (65, 12), (65, 13), (65, 14), (65, 15), (65, 16), (65, 17), (65, 18), (65, 19), (65, 20), (65, 21), (66, 8), (66, 9), (66, 10), (66, 11), (66, 12), (66, 13), (66, 14), (66, 15), (66, 16), (66, 17), (66, 18), (66, 19), (66, 20), (66, 21), (67, 8), (67, 9), (67, 10), (67, 11), (67, 12), (67, 13), (67, 14), (67, 15), (67, 16), (67, 17), (67, 18), (67, 19), (67, 20), (67, 21), (68, 8), (68, 9), (68, 10), (68, 11), (68, 12), (68, 13), (68, 14), (68, 15), (68, 16), (68, 17), (68, 18), (68, 19), (68, 20), (68, 21), (69, 8), (69, 9), (69, 10), (69, 11), (69, 12), (69, 13), (69, 14), (69, 15), (69, 16), (69, 17), (69, 18), (69, 19), (69, 20), (69, 21), (70, 8), (70, 9), (70, 10), (70, 11), (70, 12), (70, 13), (70, 14), (70, 15), (70, 16), (70, 17), (70, 18), (70, 19), (70, 20), (70, 21), (71, 8), (71, 9), (71, 10), (71, 11), (71, 12), (71, 13), (71, 14), (71, 15), (71, 16), (71, 17), (71, 18), (71, 19), (71, 20), (71, 21), (72, 8), (72, 9), (72, 10), (72, 11), (72, 12), (72, 13), (72, 14), (72, 15), (72, 16), (72, 17), (72, 18), (72, 19), (72, 20), (72, 21), (73, 8), (73, 9), (73, 10), (73, 11), (73, 12), (73, 13), (73, 14), (73, 15), (73, 16), (73, 17), (73, 18), (73, 19), (73, 20), (73, 21), (74, 8), (74, 9), (74, 10), (74, 11), (74, 12), (74, 13), (74, 14), (74, 15), (74, 16), (74, 17), (74, 18), (74, 19), (74, 20), (74, 21), (75, 8), (75, 9), (75, 10), (75, 11), (75, 12), (75, 13), (75, 14), (75, 15), (75, 16), (75, 17), (75, 18), (75, 19), (75, 20), (75, 21), (76, 8), (76, 9), (76, 10), (76, 11), (76, 12), (76, 13), (76, 14), (76, 15), (76, 16), (76, 17), (76, 18), (76, 19), (76, 20), (76, 21), (77, 8), (77, 9), (77, 10), (77, 11), (77, 12), (77, 13), (77, 14), (77, 15), (77, 16), (77, 17), (77, 18), (77, 19), (77, 20), (77, 21), (78, 8), (78, 9), (78, 10), (78, 11), (78, 12), (78, 13), (78, 14), (78, 15), (78, 16), (78, 17), (78, 18), (78, 19), (78, 20), (78, 21), (79, 8), (79, 9), (79, 10), (79, 11), (79, 12), (79, 13), (79, 14), (79, 15), (79, 16), (79, 17), (79, 18), (79, 19), (79, 20), (79, 21), (80, 8), (80, 9), (80, 10), (80, 11), (80, 12), (80, 13), (80, 14), (80, 15), (80, 16), (80, 17), (80, 18), (80, 19), (80, 20), (80, 21), (81, 8), (81, 9), (81, 10), (81, 11), (81, 12), (81, 13), (81, 14), (81, 15), (81, 16), (81, 17), (81, 18), (81, 19), (81, 20), (81, 21), (82, 8), (82, 9), (82, 10), (82, 11), (82, 12), (82, 13), (82, 14), (82, 15), (82, 16), (82, 17), (82, 18), (82, 19), (82, 20), (82, 21), (83, 8), (83, 9), (83, 10), (83, 11), (83, 12), (83, 13), (83, 14), (83, 15), (83, 16), (83, 17), (83, 18), (83, 19), (83, 20), (83, 21), (84, 8), (84, 9), (84, 10), (84, 11), (84, 12), (84, 13), (84, 14), (84, 15), (84, 16), (84, 17), (84, 18), (84, 19), (84, 20), (84, 21),
                         (85, 12), (85, 13), (85, 14), (85, 15), (85, 16), (85, 17), (85, 18), (85, 19), (85, 20), (85, 21), (86, 12), (86, 13), (86, 14), (86, 15), (86, 16), (86, 17), (86, 18), (86, 19), (86, 20), (86, 21), (87, 12), (87, 13), (87, 14), (87, 15), (87, 16), (87, 17), (87, 18), (87, 19), (87, 20), (87, 21), (88, 12), (88, 13), (88, 14), (88, 15), (88, 16), (88, 17), (88, 18), (88, 19), (88, 20), (88, 21), (89, 12), (89, 13), (89, 14), (89, 15), (89, 16), (89, 17), (89, 18), (89, 19), (89, 20), (89, 21), (90, 12), (90, 13), (90, 14), (90, 15), (90, 16), (90, 17), (90, 18), (90, 19), (90, 20), (90, 21), (91, 12), (91, 13), (91, 14), (91, 15), (91, 16), (91, 17), (91, 18), (91, 19), (91, 20), (91, 21), (92, 12), (92, 13), (92, 14), (92, 15), (92, 16), (92, 17), (92, 18), (92, 19), (92, 20), (92, 21),
                         (93, 8), (93, 9), (93, 10), (93, 11), (93, 12), (93, 13), (93, 14), (93, 15), (93, 16), (93, 17), (93, 18), (93, 19), (93, 20), (93, 21), (94, 8), (94, 9), (94, 10), (94, 11), (94, 12), (94, 13), (94, 14), (94, 15), (94, 16), (94, 17), (94, 18), (94, 19), (94, 20), (94, 21), (95, 8), (95, 9), (95, 10), (95, 11), (95, 12), (95, 13), (95, 14), (95, 15), (95, 16), (95, 17), (95, 18), (95, 19), (95, 20), (95, 21), (96, 8), (96, 9), (96, 10), (96, 11), (96, 12), (96, 13), (96, 14), (96, 15), (96, 16), (96, 17), (96, 18), (96, 19), (96, 20), (96, 21), (97, 8), (97, 9), (97, 10), (97, 11), (97, 12), (97, 13), (97, 14), (97, 15), (97, 16), (97, 17), (97, 18), (97, 19), (97, 20), (97, 21), (98, 8), (98, 9), (98, 10), (98, 11), (98, 12), (98, 13), (98, 14), (98, 15), (98, 16), (98, 17), (98, 18), (98, 19), (98, 20), (98, 21), (99, 8), (99, 9), (99, 10), (99, 11), (99, 12), (99, 13), (99, 14), (99, 15), (99, 16), (99, 17), (99, 18), (99, 19), (99, 20), (99, 21), (100, 8), (100, 9), (100, 10), (100, 11), (100, 12), (100, 13), (100, 14), (100, 15), (100, 16), (100, 17), (100, 18), (100, 19), (100, 20), (100, 21),
                         (101, 8), (101, 9), (101, 10), (101, 11), (101, 12), (101, 13), (101, 14), (101, 15), (102, 8), (102, 9), (102, 10), (102, 11), (102, 12), (102, 13), (102, 14), (102, 15), (103, 8), (103, 9), (103, 10), (103, 11), (103, 12), (103, 13), (103, 14), (103, 15), (104, 8), (104, 9), (104, 10), (104, 11), (104, 12), (104, 13), (104, 14), (104, 15), (105, 8), (105, 9), (105, 10), (105, 11), (105, 12), (105, 13), (105, 14), (105, 15), (106, 8), (106, 9), (106, 10), (106, 11), (106, 12), (106, 13), (106, 14), (106, 15), (107, 8), (107, 9), (107, 10), (107, 11), (107, 12), (107, 13), (107, 14), (107, 15), (108, 8), (108, 9), (108, 10), (108, 11), (108, 12), (108, 13), (108, 14), (108, 15),
                         (109, 8), (109, 9), (109, 10), (109, 11), (109, 12), (109, 13), (109, 14), (109, 15), (109, 16), (109, 17), (109, 18), (109, 19), (110, 8), (110, 9), (110, 10), (110, 11), (110, 12), (110, 13), (110, 14), (110, 15), (110, 16), (110, 17), (110, 18), (110, 19), (111, 8), (111, 9), (111, 10), (111, 11), (111, 12), (111, 13), (111, 14), (111, 15), (111, 16), (111, 17), (111, 18), (111, 19), (112, 8), (112, 9), (112, 10), (112, 11), (112, 12), (112, 13), (112, 14), (112, 15), (112, 16), (112, 17), (112, 18), (112, 19), (113, 8), (113, 9), (113, 10), (113, 11), (113, 12), (113, 13), (113, 14), (113, 15), (113, 16), (113, 17), (113, 18), (113, 19), (114, 8), (114, 9), (114, 10), (114, 11), (114, 12), (114, 13), (114, 14), (114, 15), (114, 16), (114, 17), (114, 18), (114, 19), (115, 8), (115, 9), (115, 10), (115, 11), (115, 12), (115, 13), (115, 14), (115, 15), (115, 16), (115, 17), (115, 18), (115, 19), (116, 8), (116, 9), (116, 10), (116, 11), (116, 12), (116, 13), (116, 14), (116, 15), (116, 16), (116, 17), (116, 18), (116, 19), (117, 8), (117, 9), (117, 10), (117, 11), (117, 12), (117, 13), (117, 14), (117, 15), (117, 16), (117, 17), (117, 18), (117, 19), (118, 8), (118, 9), (118, 10), (118, 11), (118, 12), (118, 13), (118, 14), (118, 15), (118, 16), (118, 17), (118, 18), (118, 19),
                         (119, 11), (119, 12), (119, 13), (119, 14), (119, 15), (119, 16), (119, 17), (119, 18), (120, 11), (120, 12), (120, 13), (120, 14), (120, 15), (120, 16), (120, 17), (120, 18), (121, 11), (121, 12), (121, 13), (121, 14), (121, 15), (121, 16), (121, 17), (121, 18), (122, 11), (122, 12), (122, 13), (122, 14), (122, 15), (122, 16), (122, 17), (122, 18), (123, 11), (123, 12), (123, 13), (123, 14), (123, 15), (123, 16), (123, 17), (123, 18), (124, 11), (124, 12), (124, 13), (124, 14), (124, 15), (124, 16), (124, 17), (124, 18), (125, 11), (125, 12), (125, 13), (125, 14), (125, 15), (125, 16), (125, 17), (125, 18), (126, 11), (126, 12), (126, 13), (126, 14), (126, 15), (126, 16), (126, 17), (126, 18), (127, 11), (127, 12), (127, 13), (127, 14), (127, 15), (127, 16), (127, 17), (127, 18), (128, 11), (128, 12), (128, 13), (128, 14), (128, 15), (128, 16), (128, 17), (128, 18), (129, 11), (129, 12), (129, 13), (129, 14), (129, 15), (129, 16), (129, 17), (129, 18), (130, 11), (130, 12), (130, 13), (130, 14), (130, 15), (130, 16), (130, 17), (130, 18), (131, 11), (131, 12), (131, 13), (131, 14), (131, 15), (131, 16), (131, 17), (131, 18), (132, 11), (132, 12), (132, 13), (132, 14), (132, 15), (132, 16), (132, 17), (132, 18), (133, 11), (133, 12), (133, 13), (133, 14), (133, 15), (133, 16), (133, 17), (133, 18), (134, 11), (134, 12), (134, 13), (134, 14), (134, 15), (134, 16), (134, 17), (134, 18), (135, 11), (135, 12), (135, 13), (135, 14), (135, 15), (135, 16), (135, 17), (135, 18), (136, 11), (136, 12), (136, 13), (136, 14), (136, 15), (136, 16), (136, 17), (136, 18), (137, 11), (137, 12), (137, 13), (137, 14), (137, 15), (137, 16), (137, 17), (137, 18), (138, 11), (138, 12), (138, 13), (138, 14), (138, 15), (138, 16), (138, 17), (138, 18), (139, 11), (139, 12), (139, 13), (139, 14), (139, 15), (139, 16), (139, 17), (139, 18), (140, 11), (140, 12), (140, 13), (140, 14), (140, 15), (140, 16), (140, 17), (140, 18), (141, 11), (141, 12), (141, 13), (141, 14), (141, 15), (141, 16), (141, 17), (141, 18), (142, 11), (142, 12), (142, 13), (142, 14), (142, 15), (142, 16), (142, 17), (142, 18), (143, 11), (143, 12), (143, 13), (143, 14), (143, 15), (143, 16), (143, 17), (143, 18), (144, 11), (144, 12), (144, 13), (144, 14), (144, 15), (144, 16), (144, 17), (144, 18), (145, 11), (145, 12), (145, 13), (145, 14), (145, 15), (145, 16), (145, 17), (145, 18), (146, 11), (146, 12), (146, 13), (146, 14), (146, 15), (146, 16), (146, 17), (146, 18), (147, 11), (147, 12), (147, 13), (147, 14), (147, 15), (147, 16), (147, 17), (147, 18), (148, 11), (148, 12), (148, 13), (148, 14), (148, 15), (148, 16), (148, 17), (148, 18), (149, 11), (149, 12), (149, 13), (149, 14), (149, 15), (149, 16), (149, 17), (149, 18), (150, 11), (150, 12), (150, 13), (150, 14), (150, 15), (150, 16), (150, 17), (150, 18), (151, 11), (151, 12), (151, 13), (151, 14), (151, 15), (151, 16), (151, 17), (151, 18), (152, 11), (152, 12), (152, 13), (152, 14), (152, 15), (152, 16), (152, 17), (152, 18),
                         (153, 12), (153, 13), (153, 14), (153, 15), (153, 16), (153, 17), (154, 12), (154, 13), (154, 14), (154, 15), (154, 16), (154, 17), (155, 12), (155, 13), (155, 14), (155, 15), (155, 16), (155, 17), (156, 12), (156, 13), (156, 14), (156, 15), (156, 16), (156, 17)]  # Replace with your coordinates

#coordinates2 = [(x - 1, y - 1) for x, y in wrong_coordinates]
#print(coordintes2)

IndentationError: unexpected indent (1197479183.py, line 2)